In [1]:
import os
import sys
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, ZeroPadding2D, Convolution2D, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.models import model_from_json
from sklearn.metrics import f1_score
import numpy as np
from PIL import Image
import math

from predictions import *
from im_postprocess import*
from im_processing import *
from helpers import *

Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
NUM_CHANNELS    = 3  # RGB images
PIXEL_DEPTH     = 255
NUM_LABELS      = 2
TRAINING_SIZE   = 50
VALIDATION_SIZE = 5  # Size of the validation set.
SEED            = 66478  # Set to None for random seed.
BATCH_SIZE      = 16  # 64
NUM_EPOCHS      = 100
RESTORE_MODEL   = False  # If True, restore existing model instead of training a new one
RECORDING_STEP  = 0
FG_THRESH       = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch
IMG_PATCH_SIZE  = 16 # IMG_PATCH_SIZE should be a multiple of 4, image size should be an integer multiple of this number

In [5]:
data_dir = '../Datasets/training/'
train_data_filename = data_dir + 'images/'
train_labels_filename = data_dir + 'groundtruth/'

# Extract it into numpy arrays.
train_data = extract_data(train_data_filename, TRAINING_SIZE)
train_labels = extract_labels(train_labels_filename, TRAINING_SIZE)

Loading ../Datasets/training/images/satImage_001.png
Loading ../Datasets/training/images/satImage_002.png
Loading ../Datasets/training/images/satImage_003.png
Loading ../Datasets/training/images/satImage_004.png
Loading ../Datasets/training/images/satImage_005.png
Loading ../Datasets/training/images/satImage_006.png
Loading ../Datasets/training/images/satImage_007.png
Loading ../Datasets/training/images/satImage_008.png
Loading ../Datasets/training/images/satImage_009.png
Loading ../Datasets/training/images/satImage_010.png
Loading ../Datasets/training/images/satImage_011.png
Loading ../Datasets/training/images/satImage_012.png
Loading ../Datasets/training/images/satImage_013.png
Loading ../Datasets/training/images/satImage_014.png
Loading ../Datasets/training/images/satImage_015.png
Loading ../Datasets/training/images/satImage_016.png
Loading ../Datasets/training/images/satImage_017.png
Loading ../Datasets/training/images/satImage_018.png
Loading ../Datasets/training/images/satImage_0

In [6]:
#Test model simple

# Create model
# https://keras.io/models/sequential/
mod = Sequential()

# Add a layer
# https://keras.io/layers/about-keras-layers/
# https://keras.io/activations/
mod.add(Conv2D(64, kernel_size=IMG_PATCH_SIZE, activation='relu',input_shape=(IMG_PATCH_SIZE, IMG_PATCH_SIZE, NUM_CHANNELS)))
mod.add(Flatten())
mod.add(Dense(2, activation='softmax'))
        
        
# https://keras.io/optimizers/
# https://keras.io/losses/
mod.compile(optimizer='sgd', loss='mean_squared_error', metrics=['accuracy']) #f1 = 0.43
#mod.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy']) #f1 = 0.43

In [7]:
# VGG qui marche pas
mod_vgg = Sequential()
mod_vgg.add(ZeroPadding2D((1,1),input_shape=(3,224,224)))
mod_vgg.add(Convolution2D(64, 3, 3, activation="relu"))
mod_vgg.add(ZeroPadding2D((1,1)))
mod_vgg.add(Convolution2D(64, 3, 3, activation="relu"))
mod_vgg.add(MaxPooling2D((2,2), strides=(2,2), padding="same"))

mod_vgg.add(ZeroPadding2D((1,1)))
mod_vgg.add(Convolution2D(128, 3, 3, activation="relu"))
mod_vgg.add(ZeroPadding2D((1,1)))
mod_vgg.add(Convolution2D(128, 3, 3, activation="relu"))
mod_vgg.add(MaxPooling2D((2,2), strides=(2,2), padding='same')) ###This line gives error

mod_vgg.add(ZeroPadding2D((1,1)))
mod_vgg.add(Convolution2D(256, 3, 3, activation="relu"))
mod_vgg.add(ZeroPadding2D((1,1)))
mod_vgg.add(Convolution2D(256, 3, 3, activation="relu"))
mod_vgg.add(ZeroPadding2D((1,1)))
mod_vgg.add(Convolution2D(256, 3, 3, activation="relu"))
mod_vgg.add(MaxPooling2D((2,2), strides=(2,2), padding='same'))

mod_vgg.add(ZeroPadding2D((1,1)))
mod_vgg.add(Convolution2D(512, 3, 3, activation="relu"))
mod_vgg.add(ZeroPadding2D((1,1)))
mod_vgg.add(Convolution2D(512, 3, 3, activation="relu"))
mod_vgg.add(ZeroPadding2D((1,1)))
mod_vgg.add(Convolution2D(512, 3, 3, activation="relu"))
mod_vgg.add(MaxPooling2D((2,2), strides=(2,2), padding='same'))

mod_vgg.add(ZeroPadding2D((1,1)))
mod_vgg.add(Convolution2D(512, 3, 3, activation="relu"))
mod_vgg.add(ZeroPadding2D((1,1)))
mod_vgg.add(Convolution2D(512, 3, 3, activation="relu"))
mod_vgg.add(ZeroPadding2D((1,1)))
mod_vgg.add(Convolution2D(512, 3, 3, activation="relu"))
mod_vgg.add(MaxPooling2D((2,2), strides=(2,2), padding='same'))

mod_vgg.add(Flatten())
mod_vgg.add(Dense(4096, activation="relu"))
mod_vgg.add(Dropout(0.5))
mod_vgg.add(Dense(4096, activation="relu"))
mod_vgg.add(Dropout(0.5))
mod_vgg.add(Dense(1000, activation='softmax'))


#adam_optimizer = Adam(lr=0.001)
mod_vgg.compile(optimizer='sgd', loss='mean_squared_error', metrics=['accuracy'])

/Users/leobouraux/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  after removing the cwd from sys.path.
/Users/leobouraux/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  
/Users/leobouraux/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  # Remove the CWD from sys.path while we load stuff.
/Users/leobouraux/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  if sys.path[0] == '':
/Users/leobouraux/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/Users/leobouraux/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu")`
/Users/leobouraux/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu")`
/Users/leobouraux/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu")`
/Users/leobouraux/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu")`
/Users/leobouraux/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu")`
/Users/leobouraux/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarn

In [9]:
TEST_SIZE = 0.33 # To be studied

# Step 0: Shuffle samples
np.random.seed(0)
np.random.shuffle(train_data)
# resetting the seed allows for an identical shuffling between y and x
np.random.seed(0)
np.random.shuffle(train_labels)

# Step 1: Split into validation and training set     
data_tr, data_te, label_tr, label_te = train_test_split(train_data, train_labels, test_size=TEST_SIZE, random_state=1)

# Step 2: Give weights to classes ?
c_weight = {1: 3., 
            0: 1.}

# Step 3: Greate Generators
datagen_tr = ImageDataGenerator()
datagen_te = ImageDataGenerator()

train_generator = datagen_tr.flow(data_tr, label_tr, batch_size=BATCH_SIZE)
test_generator  = datagen_te.flow(data_te, label_te, batch_size=BATCH_SIZE)

# Step 4: Early stop
early_stop_callback = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=0, mode='max', restore_best_weights=True)



# Step 5: Training
mod_vgg.fit_generator(train_generator,
            validation_data=test_generator,
            steps_per_epoch=math.ceil(TRAINING_SIZE / BATCH_SIZE), #len(train_generator)/16,
            epochs=NUM_EPOCHS,
            callbacks = [early_stop_callback],
            class_weight=c_weight,
            validation_steps=math.ceil(len(test_generator)/BATCH_SIZE))




Epoch 1/100


ValueError: Error when checking input: expected zero_padding2d_1_input to have shape (3, 224, 224) but got array with shape (16, 16, 3)

In [ ]:
prediction_training_dir = "predictions_training_vgg_model/"
if not os.path.isdir(prediction_training_dir):
    os.mkdir(prediction_training_dir)

for i in range(1, TRAINING_SIZE+1):
    pimg = get_prediction_with_groundtruth(train_data_filename, i, mod_vgg)
    Image.fromarray(pimg).save(prediction_training_dir + "prediction_" + str(i) + ".png")
    oimg = get_prediction_with_overlay(train_data_filename, i, mod_vgg)
    oimg.save(prediction_training_dir + "overlay_" + str(i) + ".png")


# serialize model to JSON
OUTPUT_FILENAME = "simple_model"
model_json = mod.to_json()
with open(OUTPUT_FILENAME + ".json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
mod.save_weights(OUTPUT_FILENAME + ".h5")
print("Saved model to disk")

validation_data_prediction = mod.predict_classes(data_te)
validation_label = []
for e in label_te:
    if (e[0] == 0):
        validation_label.append(1)
    else:
        validation_label.append(0)
        
validation_label = np.array(validation_label)     
print("F1 score = "+str(f1_score(validation_data_prediction, validation_label, labels=['1'])))